In [2]:
import Levenshtein
import wikitextparser as wtp
from tqdm import tqdm
from wiki_dump_reader import iterate

from extract_books_from_DB import *

Chargement des livres depuis les bases de données

In [5]:
all_books = generate_all_books()

ADP_loading_time:  10.54570746421814
DL_loading_time:  41.25322675704956
ILE_loading time:  3.8861887454986572
Hurtubise_loading time:  0.0596621036529541
Babelio_loading_time time:  0.1908266544342041


In [6]:
with open('all_books.json', 'r') as input_file:
    all_books = json.load(input_file, cls=BookJSONDecoder)

Regroupement des livres par auteur. On utilise les noms des auteurs normalisés comme clef dans un dictionnaire
où les valeurs sont les livres dont ils sont l'auteur

In [25]:
# On recupère les données sauvegardées

author_ls = {}

for book in tqdm(all_books, total=len(all_books)):
    for author in book.authors:
        if author not in author_ls:
            author_ls[author] = [book]
        else:
            author_ls[author].append(book)

with open('authors_ls.json', 'w') as outfile:
    json.dump(author_ls, outfile, cls=BookJSONEncoder)

famous = ['Michel Tremblay', 'Anne Hébert', 'Gabrielle Roy', 'Marie Cardinal', 'Réjean Ducharme',
                'Jacques Ferron', 'Victor-Lévy Beaulieu', 'Marcel Dubé', 'Yves Thériault', 'Jacques Poulin',
                'André Langevin']

100%|██████████| 101581/101581 [00:00<00:00, 690430.01it/s]


On selectionne maitenant parmis le dump wikipédia les pages qui concernent des livres ou des auteurs

In [17]:
book_to_keep = {}
writer_to_keep = {}
count_author = 0
count_book = 0

for title, text in tqdm(iterate('./Wikipedia/frwiki-20200401-pages-articles-multistream.xml'), total=20200401):

    is_book_re = re.search(r"Infobox (Ouvrage|Livre)", text)
    is_ebauche_re = re.search(r"{{Ébauche\|livre}}", text)
    if is_book_re or is_ebauche_re:
        book_to_keep[title] = text
        count_book += 1

    is_writer_re = re.search(r"Infobox Écrivain", text)
    is_Ecrivain_Quebecois_re = re.search(r"Catégorie:Écrivain", text)
    if is_writer_re or is_Ecrivain_Quebecois_re:
        writer_to_keep[title] = text
        count_author += 1

print('count_books: ', count_book)
with open('./Wikipedia/fr_dumps_wikipedia_books.json', 'w') as outfile:
    json.dump(book_to_keep, outfile)
print('count_authors: ', count_author)
with open('./Wikipedia/fr_dumps_wikipedia_writers.json', 'w') as outfile:
    json.dump(writer_to_keep, outfile)

 19%|█▉        | 3799564/20200401 [19:20<1:23:28, 3274.70it/s]


count_books:  16749
count_authors:  40625


On recherche maintenant les auteurs et livres en communs avec la base de donnée

In [24]:
couple_books_4 = []
couple_books_6 = []
couple_books_8 = []

data_base_stats_4 = {}
data_base_stats_6 = {}
data_base_stats_8 = {}

found_at_least_once = 0
nb_couple = 0

count_freq = {}

with open('./Wikipedia/fr_dumps_wikipedia_books.json', 'r') as input_file:
    wiki_books = json.load(input_file)

with open('all_books.json', 'r') as input_file:
    all_books = json.load(input_file, cls=BookJSONDecoder)

for wiki_book, wiki_text in tqdm(wiki_books.items()):
    found = False
    count_freq_i = 0
    normalized_wiki_book = normalize(wiki_book)
    for DB_book in all_books:
        dist_auteur = Levenshtein.distance(DB_book.title, normalized_wiki_book)
        if dist_auteur < max(1, min(len(DB_book.title), len(normalized_wiki_book)) / 4):
            new_couple = {
                        'book DB': DB_book,
                        'book wiki': {'title': normalized_wiki_book, 'title_raw':wiki_book, 'text': wiki_text}
                    }
            nb_couple += 1
            count_freq_i += 1
            found = True

            if dist_auteur < max(1, min(len(DB_book.title), len(wiki_book)) / 8):
                    couple_books_8.append(new_couple)
                    try:
                        data_base_stats_8[DB_book.data_base] += 1
                    except:
                        data_base_stats_8[DB_book.data_base] = 1
            elif dist_auteur < max(1, min(len(DB_book.title), len(wiki_book)) / 6):
                couple_books_6.append(new_couple)
                try:
                    data_base_stats_6[DB_book.data_base] += 1
                except:
                    data_base_stats_6[DB_book.data_base] = 1
            else:
                couple_books_4.append(new_couple)
                try:
                    data_base_stats_4[DB_book.data_base] += 1
                except:
                    data_base_stats_4[DB_book.data_base] = 1
    if found:
        found_at_least_once += 1
        count_freq[wiki_book] = count_freq_i

print("nombre d'autheurs en commun: ", found_at_least_once, " soit ", found_at_least_once * 100 / len(wiki_books), "%")
print("nombre d'autheurs reconnus: ", nb_couple, " soit ", nb_couple/found_at_least_once, ' livre dans nos bases de donnée par livre reconnu dans wikipédia')
print("Frequence de reconnaissance: ", sorted(count_freq.items(), key=lambda item: item[1], reverse=True)[:10])

print("repartition dans les bases de données: ")
print('1/8: ', json.dumps(data_base_stats_8))
print('1/6: ', json.dumps(data_base_stats_6))
print('1/4: ', json.dumps(data_base_stats_4))

with open('Wikipedia/couple_books_wikipedia_4.json', 'w') as outfile:
    json.dump(couple_books_4, outfile, cls=BookJSONEncoder)

with open('Wikipedia/couple_books_wikipedia_6.json', 'w') as outfile:
    json.dump(couple_books_6, outfile, cls=BookJSONEncoder)

with open('Wikipedia/couple_books_wikipedia_8.json', 'w') as outfile:
    json.dump(couple_books_8, outfile, cls=BookJSONEncoder)

100%|██████████| 16749/16749 [44:35<00:00,  6.26it/s] 


nombre d'autheurs en commun:  3853  soit  3.79303216152627 %
nombre d'autheurs reconnus:  13116  soit  3.4041007007526605  livre dans nos bases de donnée par livre reconnu dans wikipédia
Frequence de reconnaissance:  [('Maria Chapdelaine', 92), ('Le Petit Chaperon rouge', 38), ('Les Anciens Canadiens', 38), ("Les Chevaliers d'Émeraude", 35), ('Poems (Agatha Christie)', 35), ('Un homme et son péché (roman)', 34), ('La Reine des anges', 31), ('La Reine des neiges (roman)', 30), ('La Reine des neiges', 30), ('Agaguk', 28)]
repartition dans les bases de données: 
1/8:  {"Depot_legal": 2852, "ADP": 313, "ILE": 1396, "Babelio": 106}
1/6:  {"Depot_legal": 700, "ILE": 320, "ADP": 83, "Babelio": 14}
1/4:  {"Depot_legal": 4661, "ILE": 2028, "ADP": 498, "Babelio": 145}


In [5]:
couple_writers_4 = []
couple_writers_6 = []
couple_writers_8 = []

data_base_stats_4 = {}
data_base_stats_6 = {}
data_base_stats_8 = {}

found_at_least_once = 0
nb_couple = 0

count_freq = {}

with open('./Wikipedia/fr_dumps_wikipedia_writers.json', 'r') as input_file:
    wiki_authors = json.load(input_file)

with open('authors_ls.json', 'r') as input_file:
    author_ls = json.load(input_file, cls=BookJSONDecoder)

for wiki_author, wiki_author_text in tqdm(wiki_authors.items()):
    found = False
    count_freq_i = 0
    normalized_wiki_author = normalize_author(wiki_author)
    for DB_author, DB_author_books in author_ls.items():
        dist_auteur = Levenshtein.distance(DB_author, normalized_wiki_author)

        if dist_auteur < max(1, min(len(DB_author), len(normalized_wiki_author)) / 4):
            new_couple = {
                        'author_DB': {'name': DB_author, 'books': DB_author_books},
                        'author_wiki': {'name': normalized_wiki_author, 'name_raw': wiki_author, 'text':wiki_author_text}
                    }

            nb_couple += 1
            count_freq_i += 1
            found = True

            if dist_auteur < max(1, min(len(DB_author), len(normalized_wiki_author)) / 8):
                    couple_writers_8.append(new_couple)
                    for book in DB_author_books:
                        try:
                            data_base_stats_8[book.data_base] += 1
                        except:
                            data_base_stats_8[book.data_base] = 1
            elif dist_auteur < max(1, min(len(DB_author), len(normalized_wiki_author)) / 6):
                couple_writers_6.append(new_couple)
                for book in DB_author_books:
                    try:
                        data_base_stats_6[book.data_base] += 1
                    except:
                        data_base_stats_6[book.data_base] = 1
            else:
                couple_writers_4.append(new_couple)
                for book in DB_author_books:
                    try:
                        data_base_stats_4[book.data_base] += 1
                    except:
                        data_base_stats_4[book.data_base] = 1
    if found:
        found_at_least_once += 1
        count_freq[wiki_author] = count_freq_i
    # if count_freq_i > 10:
    #     print("compte superieur a 10: ", author_DB)

print("nombre d'autheurs en commun: ", found_at_least_once, " soit ", found_at_least_once * 100 / len(wiki_authors), "%")
print("nombre d'autheurs reconnus: ", nb_couple, " soit ", nb_couple/found_at_least_once, ' auteur dans nos bases de donnée par auteur reconnu dans wikipedia')
print("Frequence de reconnaissance: ", count_freq)

print("repartition dans les bases de données: ")
print('1/8: ', json.dumps(data_base_stats_8))
print('1/6: ', json.dumps(data_base_stats_6))
print('1/4: ', json.dumps(data_base_stats_4))

with open('Wikipedia/couple_authors_wikipedia_4.json', 'w') as outfile:
    json.dump(couple_writers_4, outfile, cls=BookJSONEncoder)

with open('Wikipedia/couple_authors_wikipedia_6.json', 'w') as outfile:
    json.dump(couple_writers_6, outfile, cls=BookJSONEncoder)

with open('Wikipedia/couple_authors_wikipedia_8.json', 'w') as outfile:
    json.dump(couple_writers_8, outfile, cls=BookJSONEncoder)

100%|██████████| 40625/40625 [14:59<00:00, 45.17it/s]


nombre d'autheurs en commun:  5883  soit  14.48123076923077 %
nombre d'autheurs reconnus:  10322  soit  1.7545469998300187  auteur dans nos bases de donnée par auteur reconnu dans wikipedia
Frequence de reconnaissance:  {'Antoine de Saint-Exupéry': 1, 'Alfred de Musset': 1, 'Alain-Fournier': 3, 'Aldous Huxley': 1, 'Bernard Werber': 1, 'Carl Sagan': 1, 'Charles Péguy': 1, 'Charles Baudelaire': 1, 'Edgar Allan Poe': 1, 'Guillaume Apollinaire': 1, 'Gérard Manset': 1, 'Henry James': 2, 'Hermann Hesse': 1, 'Isaac Asimov': 1, 'Imre Kertész': 1, 'Jules Verne': 1, 'John Fowles': 1, 'Maurice Leblanc': 1, 'Mary Shelley': 1, 'Molière': 2, 'Marcel Aymé': 2, 'Michael Moore': 1, 'Oscar Wilde': 1, 'Paul Auster': 1, 'Pierre Boulle': 6, 'Stephen King': 1, 'Thích Nhất Hạnh': 1, 'Victor Hugo': 1, 'William Morris': 1, 'Guy de Maupassant': 1, 'Gustave Flaubert': 1, 'John Steinbeck': 1, 'Bram Stoker': 1, 'Wilbur Smith': 2, 'Henry David Thoreau': 1, 'Jean-Jacques Sempé': 1, 'Timothy Findley': 1, 'Théophile G

In [6]:
def format_list(page):
    """
    Prend une page wikipédia et supprime les indications de colonnes pour permettre au parseur de trouver les listes
    :param str page: page wikipédia
    :return: même page sans {{colonnes|taille=|nombre=2| ... }} tout en conservant le contenu
    """
    page = re.sub(r'\*\*', '*', page)
    page_serach_re = re.search(r'\{\{colonnes\|.*?\|\n', page, re.DOTALL)
    while page_serach_re:
        nb_bracket = 0
        start_index = page_serach_re.span()[0]
        for index, caractere in enumerate(page[start_index:]):
            if caractere == '{':
                nb_bracket += 1
            elif caractere == '}':
                nb_bracket -= 1
            if nb_bracket == 0:
                page = page[:start_index + index - 1] + page[start_index + index + 2:]
                page = re.sub(r'\{\{colonnes\|.*?\|\n', '', page)
                break
        page_serach_re = re.search(r'\{\{colonnes\|.*?\|\n', page, re.DOTALL)
    return page

def split_info_box(page):
    """
    Sépare un page wikipédia en son infobox et le reste de la page. Si il n'y a pas d'Infobox, retourne un champs vide
    pour L'infobox, et la page complete pour le reste de la page
    :param str page: page wikipédia
    :return (str, str): Infobox et contenu de la page sans l'infobox
    """
    nb_bracket = 0
    start_index = page.find("{{Infobox")
    for index, caractere in enumerate(page[start_index:]):
        if caractere == '{':
            nb_bracket += 1
        elif caractere == '}':
            nb_bracket -= 1
        if nb_bracket == 0:
            return page[:start_index + index + 1], page[start_index + index + 2:]
    return "", page

def get_info_from_infobox(raw_infos):
    """
    retourne les champs de l'infobox sous forme de dictionnaire à partir de sa chaine de caractère, ou si
    :param str raw_infos: chaine de caractère de l'infobox
    :return dict[str, str]: champs de l'infobox
    """
    infos = {}
    for info in re.split(r"\n\||\n\}", raw_infos, flags=re.DOTALL)[1:-1]:
        if len(info.split('=')) > 1:
            key, value = info.split('=')[0], info.split('=')[1]
            infos[key] = value
    return infos


def parse_section(section, title_ls, level):
    """
    Trie les informations que l'on peut récupérer à part d'une section tirée du parseur,
    et fait de même pour ses sous-sections si elles existent
    :param wikitextparser.Section section: Section de la page wikipedia
    :param set title_ls: liste des intitulés des sections
    :param int level: Niveau de la section (section principale, secondaire, tertiaire, etc...)
    :return dict[str: UNION[dict, list, int]]: informations pertinantes sur la section et ses sous-sections
    """

    section_dict = {'content': section.contents,
                    'list': [re.sub(r"\'\'", "", item) for list in section.get_lists() for item in list.items],
                    'sub_section': {},
                    'level':level
                    }

    # les deux premières sections sont toujours: "\n", section_actuelle
    if len(section.sections) > 2:
        sub_sections = section.sections[2:]
        for sub_sect in sub_sections:
            # On verifie bien qu'ils sagit de sous-sections
            if sub_sect.level == level + 1:
                title_ls.add(sub_sect.title)
                # Si il y a des sous-sections, on applique la fonction sur celles-ci
                section_dict['sub_section'][sub_sect.title] = parse_section(sub_sect, title_ls, sub_sect.level)

    return section_dict

with open('./Wikipedia/couple_authors_wikipedia_8.json', 'r') as outfile:
    couple_writers = json.load(outfile, cls=BookJSONDecoder)

# On compte les intitulés des sections pour la suite
title_dict = {}

for couple in tqdm(couple_writers):

    info_box, page = split_info_box(couple['author_wiki']['text'])

    couple['author_wiki']['info_box'] = get_info_from_infobox(info_box)

    parsed_page = wtp.parse(format_list(page))
    sections_infos = {}
    section_titles = set()

    for section in parsed_page.sections:
        if section.level <= 2:
            section_titles.add(section.title)
            sections_infos[section.title] = parse_section(section, section_titles, section.level)


    couple['author_wiki']['parsed_text'] = sections_infos

    # On met a jour le compte des intitulés des sections
    for title in section_titles:
        try:
            title_dict[title] += 1
        except KeyError:
            title_dict[title] = 1


print("titre des sections: ", json.dumps(sorted(title_dict.items(), key=lambda item: item[1], reverse=True)[:25], indent=2))

with open('Wikipedia/parsed_couple_writers.json', 'w') as outfile:
    json.dump(couple_writers, outfile, cls=BookJSONEncoder)

100%|██████████| 2412/2412 [00:12<00:00, 188.16it/s]


titre des sections:  [
  [
    null,
    2412
  ],
  [
    " Liens externes ",
    1977
  ],
  [
    " Biographie ",
    1720
  ],
  [
    " Notes et r\u00e9f\u00e9rences ",
    1423
  ],
  [
    " \u0152uvres ",
    810
  ],
  [
    " Bibliographie ",
    706
  ],
  [
    " Romans ",
    521
  ],
  [
    " \u0152uvre ",
    458
  ],
  [
    " Voir aussi ",
    417
  ],
  [
    " R\u00e9f\u00e9rences ",
    409
  ],
  [
    " Honneurs ",
    298
  ],
  [
    " Articles connexes ",
    291
  ],
  [
    " Publications ",
    223
  ],
  [
    " Th\u00e9\u00e2tre ",
    211
  ],
  [
    " Annexes ",
    208
  ],
  [
    " Po\u00e9sie ",
    189
  ],
  [
    " Nouvelles ",
    179
  ],
  [
    " Prix et distinctions ",
    176
  ],
  [
    " Filmographie ",
    165
  ],
  [
    " Essais ",
    152
  ],
  [
    " Sources ",
    147
  ],
  [
    " Distinctions ",
    141
  ],
  [
    " Notes ",
    124
  ],
  [
    "Biographie",
    116
  ],
  [
    " Recueils de nouvelles ",
    106
  ]
]


In [11]:
def extract_title_from_list(list):
    """
    Extrait le titre des livres depuis les listes d'items générée par le parseur
    2 étapes:
        - Si les données sont structurées : {{Ouvrage|titre=... }}, on récupère directement le titre
        - Si les donnée ne sont pas structurée: ''L'Effet des rayons gamma sur les vieux garçons'' (de [[Paul Zindel]]), Leméac ([[1970]])
          On doit eliminer les infornations en trop et ne garder que le titre.
    :param list[str] list: liste du parseur d'où extraire les titres
    :return list[str]: liste comportant uniquement les informations des titres
    """
    titles = []
    for item in [item.replace('\n', '') for item in list]:
        added = False

        # Si les informations sur les livres sont structurées
        meta_data_re = re.search(r"\{\{(Ouvrage|Écrit).*\}\}", item)
        if meta_data_re:
            meta_data = meta_data_re.group()
            # On selectionne le titre et le sous titre
            title_re = re.search(r'(?<=titre=).*?(?=\|)', meta_data)
            subtitle_re = re.search(r'(?<=sous-titre=).*?(?=\|)', meta_data)
            # Quand un sous-titre est présent, C'est souvent un tome d'une série. On concatène le titre et le sous-titre
            if title_re or subtitle_re:
                added = True
                if title_re and subtitle_re:
                    titles.append(normalize(title_re.group() + ' ' + subtitle_re.group()))
                elif title_re:
                    titles.append(normalize(title_re.group()))
                elif subtitle_re:
                    titles.append(normalize(subtitle_re.group()))

        # Si les informations ne sont pas structurées, ou que l'extraction a échouée
        if not added:
            # On retire les "{{ ...  }}", ce sont des méta-data
            n = 1
            while n:
                item, n = re.subn(r"\{\{.*\}\}", '', item)
            n = 1
            # On retire les balises html, ce sont des liens vers d'autres pages
            while n:
                item, n = re.subn(r"<.+?>.*?</.+?>", '', item)
            # On seletionne les élements entre [[...]]
            brackets_re = re.search(r'(?<=\[\[).*?(?=\]\])', item)
            while brackets_re:
                # Si on rencontre cette configuration: [[info1|info2|info3]], il s'agit de la même information
                # sous plusieures formes différentes. On choisi arbitrairement la dernière et on supprime les [[ et ]]
                item = re.sub(r"\[\[.*?\]\]", brackets_re.group().split('|')[-1], item, count=1)
                brackets_re = re.search(r'(?<=\[\[).*?(?=\]\])', item)
            # On retire dans l'ordre le texte entre parenthèse, les reférences, les années en début de chaine,
            # les espaces, les tirets et les ":" en début de chaine.
            item = remove_text_between_parentheses(item)
            item = re.sub(r'<ref .+?\\>', '', item)
            item = re.sub(r'^\s*?\d{4}', '', item)
            item = re.sub(r'^-\s', '', item)
            item = re.sub(r'^:', '', item)
            if normalize(item.split(',')[0]):
                titles.append(normalize(item.split(',')[0]))

    return titles

with open('Wikipedia/parsed_couple_writers.json', 'r') as couples_file:
    couple_writers = json.load(couples_file, cls=BookJSONDecoder)

with open('authors_ls.json', 'r') as input_file:
    author_ls = json.load(input_file, cls=BookJSONDecoder)

# Listes pour stoker nos couples confirmé, invalider et ceux dont on ne peut pas conclure car il n'y a pas de titres
couples_confirmed = []
couples_infirmed = []
couples_unsure = []

count_checked_by_titles = 0
count_differentiate_by_titles = 0
count_no_titles = 0
for couple in tqdm(couple_writers):
    confirmed_by_title_match = False
    confirmed_by_title_match_re = False
    book_titles = set()
    book_titles_raw = set()

    # On recupère tout les titres des livres de la page sous forme de set
    for title, section in couple['author_wiki']['parsed_text'].items():
        try:
            if re.search(r'bibliographie|Œuvres|Œuvre|théâtre|poésie|nouvelles|essais|publications|romans|roman|Littérature|Ouvrages|Filmographie', title, re.IGNORECASE):
                book_titles.update(extract_title_from_list(section['list']))
                book_titles_raw.update(section['list'])
        except KeyError:
            pass

    # On vérifie si on peut trouver des titres proches en comparant les titres parsés
    # au titres normalisés de nos bases de donnée
    book_titles_DB = [book.title for book in couple['author_DB']['books']]
    for title in book_titles_DB:
        for title_wiki in book_titles:
            dist_titles = Levenshtein.distance(title, title_wiki)
            if dist_titles < max(1, min(len(title), len(title_wiki)) / 4):
                confirmed_by_title_match = True
                break
    # En cas d'échec, on recherche le titre du livre directement dans les titres non nettoyé de la page wikipédia
    # via expression régulières
    for title in book_titles_DB:
        for title_wiki_raw in book_titles_raw:
            if re.search(normalize(title), normalize(title_wiki_raw)):
                confirmed_by_title_match_re = True
                break

    # On liste les cas possibles: 0,1,2 tests réussis
    if (not confirmed_by_title_match) and (not confirmed_by_title_match_re):
        if book_titles and book_titles_DB:
            count_differentiate_by_titles += 1
            couples_infirmed.append(couple)
        else:
            count_no_titles += 1
            couples_unsure.append(couple)
    else:
        count_checked_by_titles += 1
        couples_confirmed.append(couple)

    # On sauvegarde comment avons nous trouvé les correspondances entre les deux listes de titres
    couple['confirmed_by_title_match'] = confirmed_by_title_match
    couple['confirmed_by_title_match_re'] = confirmed_by_title_match_re

print('nombre de couple: ', len(couple_writers))
print('nombre de couples confirmés par titres: ', count_checked_by_titles)
print('nombre de couples infirmés par titres: ', count_differentiate_by_titles)
print('nombre de couples sans titres de wikipedia: ', count_no_titles)

with open('Wikipedia/couple_authors_wikipedia_confirmed.json', 'w') as outfile:
    json.dump(couples_confirmed, outfile, cls=BookJSONEncoder)

with open('Wikipedia/couple_authors_wikipedia_infirmed.json', 'w') as outfile:
    json.dump(couples_infirmed, outfile, cls=BookJSONEncoder)

with open('Wikipedia/couple_authors_wikipedia_unsure.json', 'w') as outfile:
    json.dump(couples_unsure, outfile, cls=BookJSONEncoder)

100%|██████████| 2412/2412 [00:19<00:00, 122.91it/s]


nombre de couple:  2412
nombre de couples confirmés par titres:  1574
nombre de couples infirmés par titres:  628
nombre de couples sans titres de wikipedia:  210


In [14]:
with open('Wikipedia/parsed_couple_writers.json', 'r') as input_file:
    couples_writers = json.load(input_file, cls=BookJSONDecoder)

count = 0
mean_count = []
for couple in couples_writers:
    award_titles = set()

    for title, section in couple['author_wiki']['parsed_text'].items():
        try:
            if re.search(r'prix', title, re.IGNORECASE):
                awards = extract_title_from_list(section['list'])
                award_titles.update(awards)
        except KeyError:
            pass

    if award_titles:
        count += 1
    mean_count.append(len(award_titles))
print("Nombre d'auteur dont on a récupéré des prix: ", count)
print("Nombre total de prix: ", sum(mean_count))


Nombre d'auteur dont on a récupéré des prix:  346
Nombre total de prix:  2458


In [16]:
with open('Wikipedia/parsed_couple_writers.json', 'r') as couples_file:
    wikipedia_couple_writers = json.load(couples_file)

with open('Wikidata/couple_authors_wikidata.json', 'r') as couples_file:
    wikidata_couple_writers = json.load(couples_file)
#diagramme de weinn database, wikipedia, wikidata

count = 0
couples_wikipedia_wikidata = []
for wikipedia_couple in wikipedia_couple_writers:
    for wikidata_couple in wikidata_couple_writers:
        if wikidata_couple['author_DB'] == wikipedia_couple['author_DB']:
            count += 1
print("Nombre d'auteurs en communs: ", count)

with open('couple_authors_wikipedia_wikidata.json', 'w') as outfile:
    json.dump(couples_wikipedia_wikidata, outfile)



KeyError: 'author_DB'